In [3]:
import requests
import pymongo

In [56]:
from dotenv import load_dotenv
load_dotenv() 
import os

FINNHUB_KEY = os.environ['FINNHUB_KEY']
FINNHUB_URI = os.environ['FINNHUB_URI']
GG_GEMINI_API = os.environ['GG_GEMINI_API']


In [5]:
sample_symbol = 'GWSO'

In [6]:
#save all book links to db
import pymongo
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access a database
db = client["stock_forum"]

# Access a collection
tbl_stock = db["tbl_stock"]

In [7]:
#query data from Finnhub but limit 30 APIs/second
#https://finnhub.io/docs/api/stock-symbols
def custom_query(get_url):
    print(get_url)
    try:
        r = requests.get(get_url)
        return r.json()
    except Exception as e:
       print(e)
       return r

In [ ]:
#1. Get all stock symbols
stock_list_url = FINNHUB_URI + 'stock/symbol?exchange=US&token=' + FINNHUB_KEY

#stock_list = custom_query(stock_list_url)
#print(stock_list[0])

https://finnhub.io/api/v1/stock/symbol?exchange=US&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
{'currency': 'USD', 'description': 'TERRACE ENERGY CORP', 'displaySymbol': 'TCRRF', 'figi': 'BBG000WCQW17', 'isin': None, 'mic': 'OOTC', 'shareClassFIGI': 'BBG001STBC13', 'symbol': 'TCRRF', 'symbol2': '', 'type': 'Common Stock'}


In [8]:
db_stocks = tbl_stock.find({})
print(db_stocks[0])

{'_id': ObjectId('678ab5f520c6690d5c97bd91'), 'currency': 'USD', 'description': 'SORTIS HOLDINGS INC', 'mic': 'OOTC', 'symbol': '111', 'type': 'Common Stock', 'about': 'long_text 111', 'wachers': 34567}


In [22]:
#2. save basic info to db -> 2 mins
for stock in stock_list:
    if stock['mic'].find('OTC') >= 0:    #this app is used for OTC market
        #upsert to db
        db_stock_detail = tbl_stock.find_one({'symbol': stock['symbol']})
        if db_stock_detail is None:
            #insert
            tbl_stock.insert_one({
                'symbol': stock['symbol'],
                "mic" : stock['mic']
            })
        else:
            #update
            tbl_stock.update_one({'symbol': stock['symbol']}, {'$set':{
                "mic" : stock['mic']
            }})

In [ ]:
#3 query basic details of stocks
stock_detail_url = FINNHUB_URI + 'stock/profile2?symbol='+sample_symbol+'&token=' + FINNHUB_KEY

stock_detail = custom_query(stock_detail_url)
print(stock_detail)
#industry, ipo, logo, name, capitalization, weburl

https://finnhub.io/api/v1/stock/profile2?symbol=GWSO&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
{'country': 'US', 'currency': 'USD', 'estimateCurrency': 'USD', 'exchange': 'OTC MARKETS', 'finnhubIndustry': 'Electrical Equipment', 'ipo': '2007-05-08', 'logo': 'https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/942965310296.png', 'marketCapitalization': 18.689048620689658, 'name': 'Global Warming Solutions Inc', 'phone': '16133631222', 'shareOutstanding': 16.39, 'ticker': 'GWSO', 'weburl': 'https://gwsogroup.com/'}


In [27]:
news_url = FINNHUB_URI + 'company-news?&from=2023-08-15&to=2024-08-20&symbol='+sample_symbol+'&token=' + FINNHUB_KEY

news = custom_query(news_url)
print(news)

https://finnhub.io/api/v1/company-news?&from=2023-08-15&to=2024-08-20&symbol=GWSO&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
[{'category': 'company', 'datetime': 1721237700, 'headline': 'Global Warming Solutions Inc. Provides Update on OTC Market Status Change and Reaffirms Commitment to Compliance and Innovation', 'id': 128736230, 'image': 'https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo-1200x1200.png', 'related': 'GWSO', 'source': 'Yahoo', 'summary': "Temecula, CA , July 17, 2024 (GLOBE NEWSWIRE) -- Global Warming Solutions Inc. (OTCQB: GWSO), a leader in developing innovative technologies to combat climate change, today addressed the recent change in its market status to the OTC Expert Market. This temporary change follows a delay in the filing of the company's Form 10-Q. GWSO is taking immediate and comprehensive steps to rectify the situation and ensure full compliance with regulatory requirements. Global Warming Solutions Inc. experienced", 'url': 'https://finnhu

In [ ]:
#similar companies
news_url = FINNHUB_URI + 'stock/peers?&from=2023-08-15&to=2024-08-20&symbol='+sample_symbol+'&token=' + FINNHUB_KEY

news = custom_query(news_url)
print(news)

https://finnhub.io/api/v1/stock/peers?&from=2023-08-15&to=2024-08-20&symbol=GWSO&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
['FLUX', 'OESX', 'SVT', 'DFLI', 'ADN', 'GWSO', 'ORBT', 'WATT', 'AURM', 'HLGN', '3665.TW']


In [29]:
#real time metric
news_url = FINNHUB_URI + 'stock/metric?&from=2023-08-15&to=2024-08-20&symbol='+sample_symbol+'&token=' + FINNHUB_KEY

news = custom_query(news_url)
print(news)

https://finnhub.io/api/v1/stock/metric?&from=2023-08-15&to=2024-08-20&symbol=GWSO&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
{'metric': {'10DayAverageTradingVolume': 0.01044, '13WeekPriceReturnDaily': 17.2864, '26WeekPriceReturnDaily': -68.5433, '3MonthADReturnStd': 188.10765, '3MonthAverageTradingVolume': 0.01772, '52WeekHigh': 6.14, '52WeekHighDate': '2024-02-13', '52WeekLow': 0.055, '52WeekLowDate': '2024-07-18', '52WeekPriceReturnDaily': -57.0732, '5DayPriceReturnDaily': -26.6667, 'assetTurnoverAnnual': 0.101, 'assetTurnoverTTM': None, 'beta': 2.716919, 'bookValuePerShareAnnual': 0.0551, 'bookValuePerShareQuarterly': 0.0181, 'cashFlowPerShareAnnual': -0.0359, 'cashFlowPerShareQuarterly': -0.0317, 'cashFlowPerShareTTM': -0.06595, 'cashPerSharePerShareAnnual': 0.0019, 'cashPerSharePerShareQuarterly': 0.0001, 'currentDividendYieldTTM': None, 'currentRatioAnnual': 0.5937, 'currentRatioQuarterly': 0.0028, 'dividendPerShareTTM': None, 'ebitdPerShareAnnual': -0.0255, 'ebitdPerShareTTM

In [ ]:
#not every stock has recommendation
#data -> do not polling data
news_url = FINNHUB_URI + 'quote?&from=2023-08-15&to=2024-08-20&symbol='+sample_symbol+'&token=' + FINNHUB_KEY

news = custom_query(news_url)
print(news)

https://finnhub.io/api/v1/quote?&from=2023-08-15&to=2024-08-20&symbol=GWSO&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
{'c': 1.14, 'd': 0.27, 'dp': 31.0345, 'h': 1.25, 'l': 0.9, 'o': 1, 'pc': 0.87, 't': 1736985600}


curl "GEMINI_API_KEY" \
-H 'Content-Type: application/json' \
-X POST \
-d '{
  "contents": [{
    "parts":[{"text": "Explain how AI works"}]
    }]
   }'

In [33]:
import re
import json

def extract_json(text_response):
    # This pattern matches a string that starts with '{' and ends with '}'
    pattern = r'\{[^{}]*\}'
    matches = re.finditer(pattern, text_response)
    json_objects = []
    for match in matches:
        json_str = match.group(0)
        try:
            # Validate if the extracted string is valid JSON
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except json.JSONDecodeError:
            # Extend the search for nested structures
            extended_json_str = extend_search(text_response, match.span())
            try:
                json_obj = json.loads(extended_json_str)
                json_objects.append(json_obj)
            except json.JSONDecodeError:
                # Handle cases where the extraction is not valid JSON
                continue
    if json_objects:
        return json_objects
    else:
        return None  # Or handle this case as you prefer
    
def extend_search(text, span):
    # Extend the search to try to capture nested structures
    start, end = span
    nest_count = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            nest_count += 1
        elif text[i] == '}':
            nest_count -= 1
            if nest_count == 0:
                return text[start:i+1]
    return text[start:end]

In [31]:
#Send a POST request
def post_request_gemini(text_prompt):
    gemini_url = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key='+GG_GEMINI_API
    HEADER = {'Content-Type': 'application/json'}
    json_data = {
        "contents": [
            { "parts": [
                {"text": text_prompt + " Please provide a response in a structured JSON format."}]
            }
        ]
    }

    try:
        r = requests.post(gemini_url, headers=HEADER, json=json_data)
        return r.json()
    except Exception as e:
       print(e)
       return e

In [43]:
#generate random posts for each stock
fake_comments = post_request_gemini("Generate 50 random comments for this stock symbol " + sample_symbol +". Each setence should have 30 to 50 words length.") #10 sec
raw_comments = fake_comments['candidates'][0]['content']['parts'][0]['text']
#extract json structure
json_comments = extract_json(raw_comments)

In [45]:
sample_comment = json_comments[0]['comments'][5]
sample_comment

"The management team at GWSO appears highly competent and has a demonstrably strong track record in the industry.  However, the market's perception of their strategies and execution plays a significant role in influencing the stock price."

In [46]:
fake_replies = post_request_gemini("Generate 7 random replies for this comment: " + sample_comment +". Each reply should have 10 to 15 words length.") #10 sec
raw_replies = fake_replies['candidates'][0]['content']['parts'][0]['text']
#extract json structure
json_replies = extract_json(raw_replies)

In [55]:
sample_reply = json_replies
sample_reply

[{'id': 1,
  'reply': 'True, perception often trumps reality in the volatile stock market.'},
 {'id': 2,
  'reply': 'Market sentiment is crucial; GWSO needs strong PR to shift perception.'},
 {'id': 3,
  'reply': "Competence isn't enough; successful execution and communication are key."},
 {'id': 4,
  'reply': "GWSO's stock price hinges on investor confidence and market trends."},
 {'id': 5,
  'reply': 'Long-term success depends on aligning strategy with market expectations.'},
 {'id': 6,
  'reply': 'Solid fundamentals are important, but market narrative matters even more.'},
 {'id': 7,
  'reply': 'Effective investor relations can bridge the gap between competence and perception.'}]

In [ ]:
#save comment

In [57]:
#todo generate random users
#avatar: https://i.pravatar.cc/150?u=martinkeith
fake_names = post_request_gemini("Generate 200 random people names.") #10 sec
raw_names = fake_names['candidates'][0]['content']['parts'][0]['text']
#extract json structure
json_names = extract_json(raw_names)

In [62]:
name_list = json_names[0]['people']
#save name to db
tbl_user = db['tbl_user']
for name in name_list:
    key = name.lower().replace(' ', '')
    #upsert to db
    db_user_detail = tbl_user.find_one({'key': key})
    if db_user_detail is None:
        #insert
        tbl_user.insert_one({
            'name': name,
            'key': key
        })
       